In [17]:
from pathlib import Path
from conllu import parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

notebook_path = Path().resolve()
code_folder = notebook_path.parent
treebanks_folder = code_folder / "treebanks"
print(treebanks_folder)

nltk.download('stopwords')
nltk.download('punkt')
norwegian_stop_words = set(stopwords.words("norwegian"))

/Users/liamo/Desktop/School/550/FinalProj/comp550-norwegian-dialects/treebanks


[nltk_data] Downloading package stopwords to /Users/liamo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/liamo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
def parse_conll_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        conll_data = file.read()
        return parse(conll_data)

In [19]:
#In this function we will have a nested list all_parsed_data (nested list of sentences where the outter list is a specific dialect while the inner list is the sentences per dialect)
#The second list all_parsed_data_filtered is the filtered version of the same thing
#Each element in the inner list is a token and there is meta data at the end
#Each token has associated info lemma,upos,etc
all_parsed_data = [] 
print(treebanks_folder)

for file_path in treebanks_folder.glob("*.conll"):
    parsed_data = parse_conll_file(file_path)
    all_parsed_data.append(parsed_data)


if all_parsed_data:
    first_file_parsed_data = all_parsed_data[0][1]
    serialized = first_file_parsed_data.serialize()
    print(first_file_parsed_data)
    print(serialized)
    token = first_file_parsed_data[0]
    # print(token['id'])
    # print(token['form'])
    # print(token['lemma'])
    # print(token['upos'])
else:
    all_parsed_data = []

/Users/liamo/Desktop/School/550/FinalProj/comp550-norwegian-dialects/treebanks
TokenList<frå, #, Førde, ?, metadata={text: "frå # Førde ?", segstart: "10.891", segstop: "11.983", file: "foerde_uib_05", speaker: "int1", id: "2"}>
# text = frå # Førde ?
# segstart = 10.891
# segstop = 11.983
# file = foerde_uib_05
# speaker = int1
# id = 2
1	frå	frå	prep	prep	_	0	FRAG	_	_
2	#	#	pause	pause	_	3	IK	_	_
3	Førde	Førde	subst	subst	prop	1	PUTFYLL	_	_
4	?	$?	clb	clb	<spm>	1	IP	_	_




In [20]:
# #Just some testing things not important really
# first_file_sentence_parsed_data = all_parsed_data[0][1]
# token = first_file_sentence_parsed_data[0]
# #print(token['lemma'])
# meta = first_file_sentence_parsed_data.metadata
# #print(meta)
# tokens = tokenize_norwegian(meta.get('text', ''))
# #print(tokens)

In [21]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model configuration
#code referenced here https://github.com/christianversloot/machine-learning-articles/blob/main/bidirectional-lstms-with-tensorflow-and-keras.md
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 140
loss_function = CategoricalCrossentropy()
max_sequence_length = 140
num_distinct_words = 10000
number_of_epochs = 100
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

In [22]:
word_2_int = {}
i = 2
for dialect in all_parsed_data:
    for token_list in dialect:
        sentence = token_list.metadata.get('text', '').split()
        for word in sentence:
            if word not in word_2_int:
                word_2_int[word] = i
                i += 1

In [24]:
better_data ={}

for dialect in all_parsed_data:

    dialect_name = dialect[0].metadata.get('file', '')
    better_data[dialect_name] = dialect

ls_aal = better_data['aal_uio_02'] + better_data['aal_uio_06']
ls_austevoll = better_data['austevoll_uib_01'] + better_data['austevoll_uib_04']

better_data['aal_uio_02'] = ls_aal
better_data['austevoll_uib_01'] = ls_austevoll

del better_data['aal_uio_06']
del better_data['austevoll_uib_04']

temp_data = {}
for dialect, token_list in better_data.items():
    temp_ls = []
    for token in token_list:
        sentence = token.metadata.get('text', '')
        if len(token) > 3:
            temp_ls.append(sentence)
    temp_data[dialect] = temp_ls

In [25]:
POS_2_int = {}
index = 6068
for key,values in better_data.items():
    for token_list in values:
        for word in token_list:
            if word['upos'] not in POS_2_int:
                POS_2_int[word['upos']] = index
                index += 1

In [27]:
integer_encoded = {}
for key,values in better_data.items():
    for token_list in values:
        if key not in integer_encoded:
            integer_encoded[key] = []

        new_sentence = [1]
        for word in token_list:
            new_sentence.append(word_2_int[word["form"]])
            new_sentence.append(POS_2_int[word['upos']])
        integer_encoded[key].append(new_sentence)

In [28]:
max_len = 0
for key,values in integer_encoded.items():
    for sentence in values:
        if len(sentence) > max_len:
            max_len = len(sentence)
print(max_len)

131


In [29]:
def prepare_data(num_classes, integer_encoded):
    from sklearn.model_selection import train_test_split
    x_train = []
    x_test = []

    y_train = []
    y_test = []

    if num_classes == 3:   
        dialect_mapping ={
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
        }

    elif num_classes == 6:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5
        }

    elif num_classes == 12:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5,
            'aal_uio_02': 6,
            'brandbu_uio_01_ny': 7,
            'hjartdal_uio_01': 8,
            'farsund_uib_02': 9,
            'foerde_uib_05': 10,
            'lierne_uio_01': 11
        }

    elif num_classes == 17:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5,
            'aal_uio_02': 6,
            'brandbu_uio_01_ny': 7,
            'hjartdal_uio_01': 8,
            'farsund_uib_02': 9,
            'foerde_uib_05': 10,
            'lierne_uio_01': 11,
            'fana_uib_03': 12,
            'hemsedal_uio_01': 13,
            'herad_uio_01': 14,
            'gol_uio_01': 15,
            'hoeyanger_uib_02': 16
        }

    integer_encoded = {key: integer_encoded[key] for key in dialect_mapping if key in integer_encoded}

    for key, values in integer_encoded.items():
        train_data, test_data = train_test_split(values, test_size=0.2, random_state=42)
        x_train.extend(train_data)
        x_test.extend(test_data)
        y_train.extend([dialect_mapping[key]] * len(train_data))
        y_test.extend([dialect_mapping[key]] * len(test_data))
    
    x_train = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) 
    x_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0)

    from keras.utils import to_categorical

    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, x_test, np.array(y_train), np.array(y_test)


In [30]:
def test_model(num_classes,x_train,x_test,y_train,y_test):
    #code referenced here https://github.com/christianversloot/machine-learning-articles/blob/main/bidirectional-lstms-with-tensorflow-and-keras.md
    model = Sequential()
    model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
    model.add(Bidirectional(LSTM(max_sequence_length)))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=additional_metrics)

    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=number_of_epochs,
            verbose=verbosity_mode,
            validation_split=validation_split)

    test_results = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    return model

In [31]:
x_train_3, x_test_3, y_train_3, y_test_3 = prepare_data(3, integer_encoded)
model_3 = test_model(3,x_train_3, x_test_3, y_train_3, y_test_3)
#7.19

Epoch 1/100
15/15 [==============================] - 6s 334ms/step - loss: 0.9099 - accuracy: 0.5404 - val_loss: 3.0418 - val_accuracy: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 4s 292ms/step - loss: 0.7903 - accuracy: 0.6118 - val_loss: 2.8626 - val_accuracy: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 5s 340ms/step - loss: 0.6614 - accuracy: 0.7342 - val_loss: 2.9087 - val_accuracy: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 4s 293ms/step - loss: 0.5153 - accuracy: 0.8117 - val_loss: 2.6026 - val_accuracy: 0.0000e+00
Epoch 5/100
15/15 [==============================] - 4s 289ms/step - loss: 0.4673 - accuracy: 0.8416 - val_loss: 2.9828 - val_accuracy: 0.0000e+00
Epoch 6/100
15/15 [==============================] - 4s 295ms/step - loss: 0.3751 - accuracy: 0.8632 - val_loss: 3.1747 - val_accuracy: 0.0000e+00
Epoch 7/100
15/15 [==============================] - 4s 290ms/step - loss: 0.3559 - accuracy: 0.8743 - val_loss: 2.287

In [32]:
x_train_6, x_test_6, y_train_6, y_test_6 = prepare_data(6, integer_encoded)
model_6 = test_model(6,x_train_6, x_test_6, y_train_6, y_test_6)
#11.58

Epoch 1/100
23/23 [==============================] - 8s 315ms/step - loss: 1.5523 - accuracy: 0.3349 - val_loss: 4.0816 - val_accuracy: 0.0000e+00
Epoch 2/100
23/23 [==============================] - 7s 297ms/step - loss: 1.4047 - accuracy: 0.4047 - val_loss: 4.9882 - val_accuracy: 0.0000e+00
Epoch 3/100
23/23 [==============================] - 7s 295ms/step - loss: 1.2417 - accuracy: 0.4785 - val_loss: 4.9326 - val_accuracy: 0.0000e+00
Epoch 4/100
23/23 [==============================] - 7s 298ms/step - loss: 1.0987 - accuracy: 0.5531 - val_loss: 5.1037 - val_accuracy: 0.0000e+00
Epoch 5/100
23/23 [==============================] - 7s 302ms/step - loss: 0.9736 - accuracy: 0.6345 - val_loss: 5.2868 - val_accuracy: 0.0000e+00
Epoch 6/100
23/23 [==============================] - 7s 297ms/step - loss: 0.8308 - accuracy: 0.6974 - val_loss: 5.1775 - val_accuracy: 0.0000e+00
Epoch 7/100
23/23 [==============================] - 7s 297ms/step - loss: 0.7075 - accuracy: 0.7533 - val_loss: 5.430

In [33]:
x_train_12, x_test_12, y_train_12, y_test_12 = prepare_data(12, integer_encoded)
model_12 = test_model(12,x_train_12, x_test_12, y_train_12, y_test_12)
#18.10

Epoch 1/100
34/34 [==============================] - 12s 323ms/step - loss: 2.1563 - accuracy: 0.2277 - val_loss: 6.3730 - val_accuracy: 0.0000e+00
Epoch 2/100
34/34 [==============================] - 11s 319ms/step - loss: 1.9896 - accuracy: 0.2694 - val_loss: 6.7784 - val_accuracy: 0.0000e+00
Epoch 3/100
34/34 [==============================] - 10s 309ms/step - loss: 1.8376 - accuracy: 0.3339 - val_loss: 6.5877 - val_accuracy: 0.0000e+00
Epoch 4/100
34/34 [==============================] - 10s 308ms/step - loss: 1.6292 - accuracy: 0.4171 - val_loss: 6.9196 - val_accuracy: 0.0000e+00
Epoch 5/100
34/34 [==============================] - 11s 312ms/step - loss: 1.4420 - accuracy: 0.4888 - val_loss: 7.0099 - val_accuracy: 0.0000e+00
Epoch 6/100
34/34 [==============================] - 10s 304ms/step - loss: 1.2588 - accuracy: 0.5596 - val_loss: 7.4533 - val_accuracy: 0.0000e+00
Epoch 7/100
34/34 [==============================] - 10s 308ms/step - loss: 1.0711 - accuracy: 0.6322 - val_loss

In [34]:
x_train_17, x_test_17, y_train_17, y_test_17 = prepare_data(17, integer_encoded)
model_17 = test_model(17,x_train_17, x_test_17, y_train_17, y_test_17)
#20.26

Epoch 1/100
40/40 [==============================] - 16s 362ms/step - loss: 2.4000 - accuracy: 0.1931 - val_loss: 7.2693 - val_accuracy: 0.0000e+00
Epoch 2/100
40/40 [==============================] - 13s 331ms/step - loss: 2.2228 - accuracy: 0.2266 - val_loss: 7.3456 - val_accuracy: 0.0000e+00
Epoch 3/100
40/40 [==============================] - 12s 311ms/step - loss: 2.0338 - accuracy: 0.3101 - val_loss: 7.9293 - val_accuracy: 0.0000e+00
Epoch 4/100
40/40 [==============================] - 13s 316ms/step - loss: 1.8005 - accuracy: 0.3890 - val_loss: 8.6143 - val_accuracy: 0.0000e+00
Epoch 5/100
40/40 [==============================] - 13s 323ms/step - loss: 1.5921 - accuracy: 0.4586 - val_loss: 8.6693 - val_accuracy: 0.0000e+00
Epoch 6/100
40/40 [==============================] - 13s 313ms/step - loss: 1.3969 - accuracy: 0.5186 - val_loss: 9.0818 - val_accuracy: 0.0000e+00
Epoch 7/100
40/40 [==============================] - 13s 315ms/step - loss: 1.2082 - accuracy: 0.5874 - val_loss